In [ ]:
!pip install apache-tvm==0.10.dev726

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tvm
from tvm import te
from tvm import autotvm
import numpy as np
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner

In [ ]:
M,N,K = 1024, 1024, 1024
target = "llvm -mcpu=core-avx2"

**Original matmul**

In [ ]:
def matmul(N, M, K):
    k = te.reduce_axis((0, K), "k")
    A = te.placeholder((M, K), name="A")
    B = te.placeholder((K, N), name="B")
    C = te.compute((M, N), lambda m, n: te.sum(A[m, k] * B[k, n], axis=k), name="C")
    s = te.create_schedule(C.op)
    return s, [A, B, C]
s, (A, B, C) = matmul(M, N, K)
print(tvm.lower(s, [A, B, C], simple_mode=True))


@main = primfn(A_1: handle, B_1: handle, C_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {A: Buffer(A_2: Pointer(float32), float32, [1048576], []),
             B: Buffer(B_2: Pointer(float32), float32, [1048576], []),
             C: Buffer(C_2: Pointer(float32), float32, [1048576], [])}
  buffer_map = {A_1: A, B_1: B, C_1: C}
  preflattened_buffer_map = {A_1: A_3: Buffer(A_2, float32, [1024, 1024], []), B_1: B_3: Buffer(B_2, float32, [1024, 1024], []), C_1: C_3: Buffer(C_2, float32, [1024, 1024], [])} {
  for (m: int32, 0, 1024) {
    for (n: int32, 0, 1024) {
      C[((m*1024) + n)] = 0f32
      for (k: int32, 0, 1024) {
        let cse_var_2: int32 = (m*1024)
        let cse_var_1: int32 = (cse_var_2 + n)
        C[cse_var_1] = (C[cse_var_1] + (A[(cse_var_2 + k)]*B[((k*1024) + n)]))
      }
    }
  }
}




```
@main = primfn(A_1: handle, B_1: handle, C_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {A: Buffer(A_2: Pointer(float32), float32, [1048576], []),
             B: Buffer(B_2: Pointer(float32), float32, [1048576], []),
             C: Buffer(C_2: Pointer(float32), float32, [1048576], [])}
  buffer_map = {A_1: A, B_1: B, C_1: C}
  preflattened_buffer_map = {A_1: A_3: Buffer(A_2, float32, [1024, 1024], []), B_1: B_3: Buffer(B_2, float32, [1024, 1024], []), C_1: C_3: Buffer(C_2, float32, [1024, 1024], [])} {
  for (m: int32, 0, 1024) {
    for (n: int32, 0, 1024) {
      C[((m*1024) + n)] = 0f32
      for (k: int32, 0, 1024) {
        let cse_var_2: int32 = (m*1024)
        let cse_var_1: int32 = (cse_var_2 + n)
        C[cse_var_1] = (C[cse_var_1] + (A[(cse_var_2 + k)]*B[((k*1024) + n)]))
      }
    }
  }
}
```







**matmul with knob transforms**

In [ ]:
def matmul_transforms(N, M, K):
    k = te.reduce_axis((0, K), "k")
    A = te.placeholder((M, K), name="A")
    B = te.placeholder((K, N), name="B")
    C = te.compute((M, N), lambda m, n: te.sum(A[m, k] * B[k, n], axis=k), name="C")
    s = te.create_schedule(C.op)
    # schedule
    m, n = C.op.axis
    mo, mi = s[C].split(m, factor=4)
    no, ni = s[C].split(n, factor=4)
    return s, [A, B, C]
s, (A, B, C) = matmul_transforms(M, N, K)
print(tvm.lower(s, [A, B, C], simple_mode=True))

@main = primfn(A_1: handle, B_1: handle, C_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {A: Buffer(A_2: Pointer(float32), float32, [1048576], []),
             B: Buffer(B_2: Pointer(float32), float32, [1048576], []),
             C: Buffer(C_2: Pointer(float32), float32, [1048576], [])}
  buffer_map = {A_1: A, B_1: B, C_1: C}
  preflattened_buffer_map = {A_1: A_3: Buffer(A_2, float32, [1024, 1024], []), B_1: B_3: Buffer(B_2, float32, [1024, 1024], []), C_1: C_3: Buffer(C_2, float32, [1024, 1024], [])} {
  for (m.outer: int32, 0, 256) {
    for (m.inner: int32, 0, 4) {
      for (n.outer: int32, 0, 256) {
        for (n.inner: int32, 0, 4) {
          C[((((m.outer*4096) + (m.inner*1024)) + (n.outer*4)) + n.inner)] = 0f32
          for (k: int32, 0, 1024) {
            let cse_var_3: int32 = (n.outer*4)
            let cse_var_2: int32 = ((m.outer*4096) + (m.inner*1024))
            let cse_var_1: int32 = ((cse_va

```
@main = primfn(A_1: handle, B_1: handle, C_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {A: Buffer(A_2: Pointer(float32), float32, [1048576], []),
             B: Buffer(B_2: Pointer(float32), float32, [1048576], []),
             C: Buffer(C_2: Pointer(float32), float32, [1048576], [])}
  buffer_map = {A_1: A, B_1: B, C_1: C}
  preflattened_buffer_map = {A_1: A_3: Buffer(A_2, float32, [1024, 1024], []), B_1: B_3: Buffer(B_2, float32, [1024, 1024], []), C_1: C_3: Buffer(C_2, float32, [1024, 1024], [])} {
  for (m.outer: int32, 0, 256) {
    for (m.inner: int32, 0, 4) {
      for (n.outer: int32, 0, 256) {
        for (n.inner: int32, 0, 4) {
          C[((((m.outer*4096) + (m.inner*1024)) + (n.outer*4)) + n.inner)] = 0f32
          for (k: int32, 0, 1024) {
            let cse_var_3: int32 = (n.outer*4)
            let cse_var_2: int32 = ((m.outer*4096) + (m.inner*1024))
            let cse_var_1: int32 = ((cse_var_2 + cse_var_3) + n.inner)
            C[cse_var_1] = (C[cse_var_1] + (A[(cse_var_2 + k)]*B[(((k*1024) + cse_var_3) + n.inner)]))
          }
        }
      }
    }
  }
}
```



**matmul template**

In [ ]:
@autotvm.template("matmul_template")
def matmul_template(N, M, K):
    k = te.reduce_axis((0, K), "k")
    A = te.placeholder((M, K), name="A")
    B = te.placeholder((K, N), name="B")
    C = te.compute((M, N), lambda m, n: te.sum(A[m, k] * B[k, n], axis=k), name="C")
    s = te.create_schedule(C.op)
    # schedule
    m, n = C.op.axis
    # config
    cfg = autotvm.get_config()
    candidates = [[8, 128], [16, 64], [32, 32], [64, 16], [128, 8]]
    cfg.define_split("tile_x", m, num_outputs=2, policy="candidate", candidate=candidates)
    cfg.define_split("tile_y", n, num_outputs=2, policy="candidate", candidate=candidates)
    return s, [A, B, C]
#s, (A, B, C) = matmul_template(M, N, K)
task = autotvm.task.create("matmul_template", args=(M, N, K), target=target)
print(task.config_space)
for idx in range(task.config_space.range_length):
    if task.config_space.is_index_valid(idx):
        print(task.config_space.get(idx))

ConfigSpace (len=25, range_length=25, space_map=
   0 tile_x: Split(policy=candidate, product=1024, num_outputs=2) len=5
   1 tile_y: Split(policy=candidate, product=1024, num_outputs=2) len=5
)
[('tile_x', [8, 128]), ('tile_y', [8, 128])],None,0
[('tile_x', [16, 64]), ('tile_y', [8, 128])],None,1
[('tile_x', [32, 32]), ('tile_y', [8, 128])],None,2
[('tile_x', [64, 16]), ('tile_y', [8, 128])],None,3
[('tile_x', [128, 8]), ('tile_y', [8, 128])],None,4
[('tile_x', [8, 128]), ('tile_y', [16, 64])],None,5
[('tile_x', [16, 64]), ('tile_y', [16, 64])],None,6
[('tile_x', [32, 32]), ('tile_y', [16, 64])],None,7
[('tile_x', [64, 16]), ('tile_y', [16, 64])],None,8
[('tile_x', [128, 8]), ('tile_y', [16, 64])],None,9
[('tile_x', [8, 128]), ('tile_y', [32, 32])],None,10
[('tile_x', [16, 64]), ('tile_y', [32, 32])],None,11
[('tile_x', [32, 32]), ('tile_y', [32, 32])],None,12
[('tile_x', [64, 16]), ('tile_y', [32, 32])],None,13
[('tile_x', [128, 8]), ('tile_y', [32, 32])],None,14
[('tile_x', [8, 128]



```
ConfigSpace (len=25, range_length=25, space_map=
   0 tile_x: Split(policy=candidate, product=1024, num_outputs=2) len=5
   1 tile_y: Split(policy=candidate, product=1024, num_outputs=2) len=5
)
[('tile_x', [8, 128]), ('tile_y', [8, 128])],None,0
[('tile_x', [16, 64]), ('tile_y', [8, 128])],None,1
[('tile_x', [32, 32]), ('tile_y', [8, 128])],None,2
[('tile_x', [64, 16]), ('tile_y', [8, 128])],None,3
[('tile_x', [128, 8]), ('tile_y', [8, 128])],None,4
[('tile_x', [8, 128]), ('tile_y', [16, 64])],None,5
[('tile_x', [16, 64]), ('tile_y', [16, 64])],None,6
[('tile_x', [32, 32]), ('tile_y', [16, 64])],None,7
[('tile_x', [64, 16]), ('tile_y', [16, 64])],None,8
[('tile_x', [128, 8]), ('tile_y', [16, 64])],None,9
[('tile_x', [8, 128]), ('tile_y', [32, 32])],None,10
[('tile_x', [16, 64]), ('tile_y', [32, 32])],None,11
[('tile_x', [32, 32]), ('tile_y', [32, 32])],None,12
[('tile_x', [64, 16]), ('tile_y', [32, 32])],None,13
[('tile_x', [128, 8]), ('tile_y', [32, 32])],None,14
[('tile_x', [8, 128]), ('tile_y', [64, 16])],None,15
[('tile_x', [16, 64]), ('tile_y', [64, 16])],None,16
[('tile_x', [32, 32]), ('tile_y', [64, 16])],None,17
[('tile_x', [64, 16]), ('tile_y', [64, 16])],None,18
[('tile_x', [128, 8]), ('tile_y', [64, 16])],None,19
[('tile_x', [8, 128]), ('tile_y', [128, 8])],None,20
[('tile_x', [16, 64]), ('tile_y', [128, 8])],None,21
[('tile_x', [32, 32]), ('tile_y', [128, 8])],None,22
[('tile_x', [64, 16]), ('tile_y', [128, 8])],None,23
[('tile_x', [128, 8]), ('tile_y', [128, 8])],None,24

```



**matmul template with filter**

In [ ]:
@autotvm.template("matmul_template_with_filter")
def matmul_template_with_filter(N, M, K):
    k = te.reduce_axis((0, K), "k")
    A = te.placeholder((M, K), name="A")
    B = te.placeholder((K, N), name="B")
    C = te.compute((M, N), lambda m, n: te.sum(A[m, k] * B[k, n], axis=k), name="C")
    s = te.create_schedule(C.op)
    # schedule
    m, n = C.op.axis
    # config
    cfg = autotvm.get_config()
    filter = lambda v: v.size[0] != 16 and v.size[1] != 16
    candidates = [[8, 128], [16, 64], [32, 32], [64, 16], [128, 8]]
    cfg.define_split("tile_x", m, num_outputs=2, policy="candidate", candidate=candidates, filter=filter)
    cfg.define_split("tile_y", n, num_outputs=2, policy="candidate", candidate=candidates, filter=filter)
    return s, [A, B, C]
#s, (A, B, C) = matmul_template_with_filter(M, N, K)
task = autotvm.task.create("matmul_template_with_filter", args=(M, N, K), target=target)
print(task.config_space)
for idx in range(task.config_space.range_length):
    if task.config_space.is_index_valid(idx):
        print(task.config_space.get(idx))


ConfigSpace (len=9, range_length=9, space_map=
   0 tile_x: Split(policy=candidate, product=1024, num_outputs=2) len=3
   1 tile_y: Split(policy=candidate, product=1024, num_outputs=2) len=3
)
[('tile_x', [8, 128]), ('tile_y', [8, 128])],None,0
[('tile_x', [32, 32]), ('tile_y', [8, 128])],None,1
[('tile_x', [128, 8]), ('tile_y', [8, 128])],None,2
[('tile_x', [8, 128]), ('tile_y', [32, 32])],None,3
[('tile_x', [32, 32]), ('tile_y', [32, 32])],None,4
[('tile_x', [128, 8]), ('tile_y', [32, 32])],None,5
[('tile_x', [8, 128]), ('tile_y', [128, 8])],None,6
[('tile_x', [32, 32]), ('tile_y', [128, 8])],None,7
[('tile_x', [128, 8]), ('tile_y', [128, 8])],None,8




```
ConfigSpace (len=9, range_length=9, space_map=
   0 tile_x: Split(policy=candidate, product=1024, num_outputs=2) len=3
   1 tile_y: Split(policy=candidate, product=1024, num_outputs=2) len=3
)
[('tile_x', [8, 128]), ('tile_y', [8, 128])],None,0
[('tile_x', [32, 32]), ('tile_y', [8, 128])],None,1
[('tile_x', [128, 8]), ('tile_y', [8, 128])],None,2
[('tile_x', [8, 128]), ('tile_y', [32, 32])],None,3
[('tile_x', [32, 32]), ('tile_y', [32, 32])],None,4
[('tile_x', [128, 8]), ('tile_y', [32, 32])],None,5
[('tile_x', [8, 128]), ('tile_y', [128, 8])],None,6
[('tile_x', [32, 32]), ('tile_y', [128, 8])],None,7
[('tile_x', [128, 8]), ('tile_y', [128, 8])],None,8

```



**matmul template with multi-filter**

In [ ]:
@autotvm.template("matmul_template_with_multi_filter")
def matmul_template_with_multi_filter(N, M, K):
    k = te.reduce_axis((0, K), "k")
    A = te.placeholder((M, K), name="A")
    B = te.placeholder((K, N), name="B")
    C = te.compute((M, N), lambda m, n: te.sum(A[m, k] * B[k, n], axis=k), name="C")
    s = te.create_schedule(C.op)
    # schedule
    m, n = C.op.axis
    # config
    cfg = autotvm.get_config()
    multi_filter = lambda e: 32 <= (e["tile_x"].size[1] + e["tile_y"].size[1]) < 128
    candidates = [[8, 128], [16, 64], [32, 32], [64, 16], [128, 8]]
    cfg.multi_filter(filter=multi_filter)
    cfg.define_split("tile_x", m, num_outputs=2, policy="candidate", candidate=candidates)
    cfg.define_split("tile_y", n, num_outputs=2, policy="candidate", candidate=candidates)
    return s, [A, B, C]
#s, (A, B, C) = matmul_template_with_multi_filter(M, N, K)
task = autotvm.task.create("matmul_template_with_multi_filter", args=(M, N, K), target=target)
print(task.config_space)
for idx in range(task.config_space.range_length):
    if task.config_space.is_index_valid(idx):
        print(task.config_space.get(idx))

ConfigSpace (len=12, range_length=25, space_map=
   0 tile_x: Split(policy=candidate, product=1024, num_outputs=2) len=5
   1 tile_y: Split(policy=candidate, product=1024, num_outputs=2) len=5
)
[('tile_x', [32, 32]), ('tile_y', [16, 64])],None,7
[('tile_x', [64, 16]), ('tile_y', [16, 64])],None,8
[('tile_x', [128, 8]), ('tile_y', [16, 64])],None,9
[('tile_x', [16, 64]), ('tile_y', [32, 32])],None,11
[('tile_x', [32, 32]), ('tile_y', [32, 32])],None,12
[('tile_x', [64, 16]), ('tile_y', [32, 32])],None,13
[('tile_x', [128, 8]), ('tile_y', [32, 32])],None,14
[('tile_x', [16, 64]), ('tile_y', [64, 16])],None,16
[('tile_x', [32, 32]), ('tile_y', [64, 16])],None,17
[('tile_x', [64, 16]), ('tile_y', [64, 16])],None,18
[('tile_x', [16, 64]), ('tile_y', [128, 8])],None,21
[('tile_x', [32, 32]), ('tile_y', [128, 8])],None,22




```
ConfigSpace (len=12, range_length=25, space_map=
   0 tile_x: Split(policy=candidate, product=1024, num_outputs=2) len=5
   1 tile_y: Split(policy=candidate, product=1024, num_outputs=2) len=5
)
[('tile_x', [32, 32]), ('tile_y', [16, 64])],None,7
[('tile_x', [64, 16]), ('tile_y', [16, 64])],None,8
[('tile_x', [128, 8]), ('tile_y', [16, 64])],None,9
[('tile_x', [16, 64]), ('tile_y', [32, 32])],None,11
[('tile_x', [32, 32]), ('tile_y', [32, 32])],None,12
[('tile_x', [64, 16]), ('tile_y', [32, 32])],None,13
[('tile_x', [128, 8]), ('tile_y', [32, 32])],None,14
[('tile_x', [16, 64]), ('tile_y', [64, 16])],None,16
[('tile_x', [32, 32]), ('tile_y', [64, 16])],None,17
[('tile_x', [64, 16]), ('tile_y', [64, 16])],None,18
[('tile_x', [16, 64]), ('tile_y', [128, 8])],None,21
[('tile_x', [32, 32]), ('tile_y', [128, 8])],None,22

```



**matmul template with filter and multi-filter**

In [ ]:
@autotvm.template("matmul_template_with_filter_and_multi_filter")
def matmul_template_with_filter_and_multi_filter(N, M, K):
    k = te.reduce_axis((0, K), "k")
    A = te.placeholder((M, K), name="A")
    B = te.placeholder((K, N), name="B")
    C = te.compute((M, N), lambda m, n: te.sum(A[m, k] * B[k, n], axis=k), name="C")
    s = te.create_schedule(C.op)
    # schedule
    m, n = C.op.axis
    # config
    cfg = autotvm.get_config()
    filter = lambda v: v.size[0] != 16 and v.size[1] != 16
    multi_filter = lambda e: 32 <= (e["tile_x"].size[1] + e["tile_y"].size[1]) < 128
    candidates = [[8, 128], [16, 64], [32, 32], [64, 16], [128, 8]]
    cfg.multi_filter(filter=multi_filter)
    cfg.define_split("tile_x", m, num_outputs=2, policy="candidate", candidate=candidates, filter=filter)
    cfg.define_split("tile_y", n, num_outputs=2, policy="candidate", candidate=candidates, filter=filter)
    return s, [A, B, C]
#s, (A, B, C) = matmul_template_with_filter_and_multi_filter(M, N, K)
task = autotvm.task.create("matmul_template_with_filter_and_multi_filter", args=(M, N, K), target=target)
print(task.config_space)
for idx in range(task.config_space.range_length):
    if task.config_space.is_index_valid(idx):
        print(task.config_space.get(idx))

ConfigSpace (len=3, range_length=9, space_map=
   0 tile_x: Split(policy=candidate, product=1024, num_outputs=2) len=3
   1 tile_y: Split(policy=candidate, product=1024, num_outputs=2) len=3
)
[('tile_x', [32, 32]), ('tile_y', [32, 32])],None,4
[('tile_x', [128, 8]), ('tile_y', [32, 32])],None,5
[('tile_x', [32, 32]), ('tile_y', [128, 8])],None,7




```
ConfigSpace (len=3, range_length=9, space_map=
   0 tile_x: Split(policy=candidate, product=1024, num_outputs=2) len=3
   1 tile_y: Split(policy=candidate, product=1024, num_outputs=2) len=3
)
[('tile_x', [32, 32]), ('tile_y', [32, 32])],None,4
[('tile_x', [128, 8]), ('tile_y', [32, 32])],None,5
[('tile_x', [32, 32]), ('tile_y', [128, 8])],None,7

```



**Run AutoTVM tuning**

In [ ]:
def run_tuning(tasks, measure_option, tuner="gridsearch", early_stopping=None, 
               log_filename="tuning.log", n_trial=None):
    for i, task in enumerate(tasks):
        prefix = "[Task %2d/%2d] " % (i + 1, len(tasks))

        # create tuner
        if tuner == "xgb" or tuner == "xgb-rank":
            tuner_obj = XGBTuner(task, loss_type="rank")
        elif tuner == "ga":
            tuner_obj = GATuner(task, pop_size=50)
        elif tuner == "random":
            tuner_obj = RandomTuner(task)
        elif tuner == "gridsearch":
            tuner_obj = GridSearchTuner(task)
        else:
            raise ValueError("Invalid tuner: " + tuner)

        # do tuning
        n_trial = min(n_trial or len(task.config_space), len(task.config_space))
        tuner_obj.tune(
            n_trial=n_trial or len(task.config_space),
            early_stopping=early_stopping,
            measure_option=measure_option,
            callbacks=[
                autotvm.callback.progress_bar(n_trial, prefix=prefix),
                autotvm.callback.log_to_file(log_filename),
            ],
        )

In [ ]:
log_file = "tune.log"
tuning_option = {
    "log_filename": log_file,
    "n_trial": 10,
    "tuner": "xgb",
    "early_stopping": None,
    "measure_option": autotvm.measure_option(
        builder=autotvm.LocalBuilder(),
        runner=autotvm.LocalRunner(
            number=1, repeat=10, min_repeat_ms=0, enable_cpu_cache_flush=True
        ),
    ),
}

In [ ]:
tasks = [task]
run_tuning(tasks, **tuning_option)

[Task  1/ 1]  Current/Best:    0.00/   0.00 GFLOPS | Progress: (3/3) | 32.20 s

 Done.


In [ ]:
s, (A, B, C) = matmul_template_with_filter_and_multi_filter(M, N, K)
with autotvm.apply_history_best(log_file):
    func = tvm.build(s, [A, B, C], target=target, name="matmul")

In [ ]:
dev = tvm.cpu(0)
a = np.random.uniform(size=(M, K)).astype("float32")
b = np.random.uniform(size=(K, N)).astype("float32")
c_tvm = tvm.nd.array(np.zeros((M, N), dtype="float32"), dev)
c_np = np.matmul(a, b)
a = tvm.nd.array(a)
b = tvm.nd.array(b)

In [ ]:
func(a, b, c_tvm)
np.testing.assert_equal(c_tvm.numpy(), c_np)

In [ ]:
time_f = func.time_evaluator(func.entry_name, dev, number=10)
cost = time_f(a,b,c_tvm).mean
print("func", cost*1000)